In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# === 入出力ディレクトリ設定 ===
# 入力は project_root/inputs
INPUT_DIR = os.path.join(os.getcwd(), "input")
# 出力は project_root/outputs
OUTPUT_DIR = os.path.join(os.getcwd(), "output")

# ディレクトリが無ければ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("INPUT_DIR:", INPUT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)



INPUT_DIR: /Users/okada1015/Desktop/マリサ/marisa/input
OUTPUT_DIR: /Users/okada1015/Desktop/マリサ/marisa/output



# 読み込み #


In [3]:
# 読み込み
df_reports = pd.read_csv(os.path.join(INPUT_DIR,"ASSESSMENT_REPORTS.csv")) #査定書情報（この情報に対して結果を返却する）
df_reads = pd.read_csv(os.path.join(INPUT_DIR,"ASSESSMENT_REPORT_SP_READ_HISTORIES.csv")) #ウェブ版査定書の閲覧状況
df_inquiries = pd.read_csv(os.path.join(INPUT_DIR,"ASSESSMENT_REPORT_INQUIRIES.csv")) #ウェブ版査定書でのユーザー問い合わせ状況
df_texts = pd.read_csv(os.path.join(INPUT_DIR,"ASSESSMENT_REPORT_INQUIRY_TEXTS.csv")) #ウェブ版査定書でのユーザー問い合わせテキスト


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_12451/2607194233.py:2: DtypeWarning: Columns (6,9,18,29,30,39,40,41,42,43,44,45,52,53,55,59,60,62,63,65,66,69,70,71,73,76,77,78,79,80,81,83,84,85,86,87,89,90,91,92,93,95,96,97,98,99,105,107,111,112,137,138,140,144,194,195,196,201,202,204,205,206,208,209,210,211,212,213,214,215,218,219,221,222,223,225,226,227,228,229,230,231,232,235,236,238,239,240,242,243,244,245,246,247,248,249,284,285,286,287,288,289,291,292,293,294,295,296,297,298,299,300,302,303) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reports = pd.read_csv(os.path.join(INPUT_DIR,"ASSESSMENT_REPORTS.csv")) #査定書情報（この情報に対して結果を返却する）


# 前処理 #

## assessment_reportsの絞り込み ##

In [5]:
df_reports.shape

(338003, 307)

In [6]:
# ASSESSED_ON を日付型に変換
df_reports['ASSESSED_ON'] = pd.to_datetime(df_reports['ASSESSED_ON'], errors='coerce')

# 期間を設定
start_date = pd.to_datetime("2023-02-01")
end_date = pd.to_datetime("2025-01-31")

# 上記の期間に該当するデータを抽出
df_reports_recent = df_reports[
    (df_reports['ASSESSED_ON'] >= start_date) &
    (df_reports['ASSESSED_ON'] <= end_date)
].copy()
print(df_reports_recent.shape)

#['IS_DELETED'] == 0のデータに絞る
df_reports_recent = df_reports_recent[df_reports_recent['IS_DELETED'] == 0]
print(df_reports_recent.shape)

#['OWNER_SPACE'] >= 40のデータに絞る
# PROPERTY_KINDごとにフィルタリング
df_reports_mansion = df_reports_recent[df_reports_recent['PROPERTY_KIND'] == 1]
print(df_reports_mansion.shape)
df_reports_land = df_reports_recent[df_reports_recent['PROPERTY_KIND'] == 2]
print(df_reports_land.shape)
df_reports_house = df_reports_recent[df_reports_recent['PROPERTY_KIND'] == 3]
print(df_reports_house.shape)

#マンションのデータのOWNER_SPACE_NUMの数値のみを抽出
df_reports_mansion['OWNER_SPACE_NUM'] = (
    df_reports_mansion['OWNER_SPACE']
    .astype(str)
    .str.extract(r'([0-9]+(?:\.[0-9]+)?)')
    .astype(float)
)
# OWNER_SPACE_NUM >= 40 でフィルタ
df_reports_mansion = df_reports_mansion[df_reports_mansion['OWNER_SPACE_NUM'] >= 40]

print(df_reports_mansion.shape)

df_reports_recent = pd.concat([df_reports_mansion, df_reports_land, df_reports_house], ignore_index=True)

# 件数確認
print("全件:", df_reports.shape)
print("2023/2/1～2025/1/31:", df_reports_recent.shape)

(225373, 307)
(219344, 307)
(125587, 307)
(24658, 307)
(69099, 307)
(113031, 308)


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_12451/1495198423.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports_mansion['OWNER_SPACE_NUM'] = (


全件: (338003, 307)
2023/2/1～2025/1/31: (206788, 308)


## 不要なカラムの削除 ##

In [7]:
# 件数確認
print(df_reads.shape)
print(df_inquiries.shape)
print(df_texts.shape)

# IDカラムを削除
df_reads = df_reads.drop(columns=['ID'])
df_inquiries = df_inquiries.drop(columns=['ID'])
df_texts = df_texts.drop(columns=['ID'])

#削除後の件数確認
print("削除後df_reads:", df_reads.shape)
print("削除後df_inquiries:", df_inquiries.shape)
print("削除後df_texts:", df_texts.shape)

(54993, 10)
(83097, 34)
(512, 7)
削除後df_reads: (54993, 9)
削除後df_inquiries: (83097, 33)
削除後df_texts: (512, 6)


# 各データの内容確認 #

In [8]:
print(f"df_reports:{df_reports_recent.shape}")
print(f"df_reads:{df_reads.shape}")
print(f"df_inquiries:{df_inquiries.shape}")
print(f"df_texts:{df_texts.shape}")

df_reports:(206788, 308)
df_reads:(54993, 9)
df_inquiries:(83097, 33)
df_texts:(512, 6)


In [9]:
df_reads.head(10)

,ASSESSMENT_REPORT_ID,FIRST_READ_AT,LAST_READ_AT,PREVIOUS_LAST_READ_AT,IS_SEND_FIRST_SP_OPEN,IS_SEND_SP_REOPEN,READ_COUNT,CREATED_AT,UPDATED_AT
0,93772,2023-10-20 15:14:35.000,2023-10-20 15:14:35.000,NaN,0,0,1,2023-10-20 15:31:08.000,2023-10-20 15:31:08.000
1,88578,2023-10-20 15:14:35.000,2023-10-24 17:05:55.000,2023-10-24 05:02:34.000,0,0,3,2023-10-20 15:31:08.000,2023-10-24 17:05:55.000
2,80231,2023-10-04 15:48:54.000,2023-10-04 15:48:54.000,NaN,0,0,1,2023-10-20 15:31:08.000,2023-10-20 15:31:08.000
3,80229,2023-08-01 16:22:24.000,2023-08-01 16:22:24.000,NaN,0,0,1,2023-10-20 15:31:08.000,2023-10-20 15:31:08.000
4,80227,2023-07-06 18:57:21.000,2023-07-06 18:57:21.000,NaN,0,0,1,2023-10-20 15:31:08.000,2023-10-20 15:31:08.000
5,92084,2023-10-19 11:15:49.000,2024-06-04 17:14:36.000,2023-12-13 22:27:44.000,0,1,5,2023-10-20 16:02:42.000,2024-06-04 17:14:36.000
6,33091,2023-10-13 22:05:52.000,2023-10-13 22:05:52.000,NaN,0,0,1,2023-10-20 16:02:42.000,2023-10-20 16:02:42.000
7,10742,2023-10-11 16:28:09.000,2023-10-11 16:28:09.000,NaN,0,0,1,2023-10-20 16:02:42.000,2023-10-20 16:02:42.000
8,103507,2023-10-06 18:21:24.000,2023-10-06 18:21:24.000,NaN,0,0,1,2023-10-20 16:02:42.000,2023-10-20 16:02:42.000
9,103512,2023-10-06 17:18:47.000,2023-10-06 17:18:47.000,NaN,0,0,1,2023-10-20 16:02:42.000,2023-10-20 16:02:42.000


In [10]:
df_inquiries.head(10)

,USER_ID,ASSESSMENT_REPORT_ID,PRICE_DETAIL,MEDIATION_BUY,MARKET_CONDITION,CONSULTATION_MOVE,TAX_ADVICE,SELLING_EXPENSE,FP_UPDATED_AT,FP_NOTIFY_READ_AT,...,TEL_UPDATED_AT,TEL_NOTIFY_READ_AT,MEETING_DATETIME_0,MEETING_DATETIME_1,MEETING_DATETIME_2,ONLINE_DESIRED,MTG_UPDATED_AT,MTG_NOTIFY_READ_AT,CREATED_AT,UPDATED_AT
0,1858,65351,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:16:07.000,2023-05-09 17:16:07.000
1,403,65406,1,1,0,1,1,0,2023-10-12 13:45:55.000,1,...,2023-10-11 16:29:09.000,1,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:19:21.000,2023-10-17 16:07:24.000
2,3730,65407,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:22:41.000,2023-05-09 17:22:41.000
3,3511,65410,1,1,1,1,1,1,2023-10-03 17:19:46.000,1,...,2023-05-09 17:26:15.000,1,2023-05-10 12:00:00.000,2023-05-11 11:00:00.000,2023-05-12 12:00:00.000,1,2023-05-09 17:26:44.000,1,2023-05-09 17:25:07.000,2023-10-03 17:21:00.000
4,403,65402,1,0,0,1,0,1,2023-05-09 17:27:30.000,1,...,NaN,0,2023-05-12 12:30:00.000,NaN,NaN,1,2023-05-09 17:27:40.000,1,2023-05-09 17:25:14.000,2023-05-09 17:28:07.000
5,602,65411,1,1,1,1,1,1,2023-05-10 14:28:26.000,1,...,2023-05-09 17:29:02.000,1,2023-05-11 11:00:00.000,NaN,NaN,0,2023-05-09 17:29:20.000,1,2023-05-09 17:28:20.000,2023-07-13 09:47:05.000
6,2351,64045,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:30:41.000,2023-05-09 17:30:41.000
7,2351,64046,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:30:41.000,2023-05-09 17:30:41.000
8,3729,63960,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,NaN,NaN,0,NaN,0,2023-05-09 17:41:51.000,2023-05-09 17:41:51.000
9,602,65416,1,1,1,1,1,1,2023-05-10 11:15:26.000,1,...,2023-05-10 10:59:23.000,1,2023-05-11 12:30:00.000,2023-05-13 15:00:00.000,NaN,0,2023-05-10 10:59:52.000,1,2023-05-09 17:42:50.000,2023-07-13 09:47:05.000


In [11]:
df_texts.head(10)

,USER_ID,ASSESSMENT_REPORT_ID,CONTENT,TXT_NOTIFY_READ_AT,CREATED_AT,UPDATED_AT
0,3511,65410,送信,1,2023-05-09 17:25:52.000,2023-05-09 17:27:10.000
1,3511,65410,メール通知テスト,1,2023-05-09 17:28:15.000,2023-05-09 17:30:33.000
2,403,65406,テスト,1,2023-05-09 17:29:47.000,2023-06-21 15:45:41.000
3,4050,60111,ありがとう。,1,2023-05-11 17:56:16.000,2023-05-11 17:58:55.000
4,4050,60111,テスト,1,2023-05-12 12:47:15.000,2023-05-12 12:58:15.000
5,4050,60111,テスト,1,2023-05-12 18:29:53.000,2023-05-12 18:31:09.000
6,2539,66182,テスト送信,1,2023-05-15 15:19:25.000,2023-07-09 18:48:14.000
7,3994,66873,お世話になっております。\n今からでも退去が7月末と決まっているので、オーナーチェンジではな...,1,2023-05-15 15:55:00.000,2023-06-09 09:54:31.000
8,3685,67175,よろしくです。,1,2023-05-17 03:13:36.000,2023-07-10 16:16:08.000
9,4129,67809,聞きたい\n,1,2023-05-19 15:38:02.000,2023-07-08 22:49:34.000


In [12]:
print(f"df_reports_recent:{df_reports_recent.columns.to_list()}")

df_reports_recent:['ID', 'USER_ID', 'IS_DEAL', 'PROPERTY_KIND', 'APARTMENT_ID', 'FLOOR_NUMBER', 'OWNER_SPACE', 'LAND_SPACE', 'BUILDING_SPACE', 'LAND_AND_BUILDING_SPACE', 'DIRECTION', 'ASSESSED_ON', 'CUSTOMER_NAME', 'PROPERTY_NAME', 'STAFF_NAME', 'COMPANY_DESCRIPTION', 'EMAIL', 'TEL', 'LINE_URL', 'HOMEPAGE', 'COVER_ASSESSED_ON', 'COVER_TITLE', 'TITLE_FONT_SIZE', 'COVER_CUSTOMER_NAME', 'COVER_STAFF_AND_COMPANY_NAME', 'COVER_DESCRIPTION', 'COVER_EMAIL', 'COVER_TEL', 'COVER_HOMEPAGE', 'COVER_STAFF_ROUND_IMAGE', 'COVER_STAFF_ROUND_IMAGE_COMMENT', 'ABOUT_PROPERTY_NAME', 'ABOUT_ADDRESS', 'LATITUDE', 'LONGITUDE', 'C_ID', 'ABOUT_TRAFFIC', 'ABOUT_MANAGER', 'ABOUT_LAND_PRIVILEGE', 'ABOUT_REGISTRY_GROUND', 'ABOUT_CITY_PLAN', 'ABOUT_TERRAIN', 'ABOUT_LAND_SHAPE', 'ABOUT_SETBACK_SPACE', 'ABOUT_CURRENT_STATE', 'ABOUT_RECONSTRUCT_DISABLED', 'ABOUT_USAGE_AREA', 'ABOUT_STRUCTURE', 'STRUCTURE_ID', 'ABOUT_BUILT_IN', 'ABOUT_FLOOR', 'ABOUT_BUILDING_COVERAGE', 'ABOUT_CONNECT_ROAD_STATE', 'ABOUT_CONNECT_ROAD',

In [13]:
print(f"df_reads:{df_reads.columns.to_list()}")

df_reads:['ASSESSMENT_REPORT_ID', 'FIRST_READ_AT', 'LAST_READ_AT', 'PREVIOUS_LAST_READ_AT', 'IS_SEND_FIRST_SP_OPEN', 'IS_SEND_SP_REOPEN', 'READ_COUNT', 'CREATED_AT', 'UPDATED_AT']


In [14]:
print(f"df_inquiries:{df_inquiries.columns.to_list()}")

df_inquiries:['USER_ID', 'ASSESSMENT_REPORT_ID', 'PRICE_DETAIL', 'MEDIATION_BUY', 'MARKET_CONDITION', 'CONSULTATION_MOVE', 'TAX_ADVICE', 'SELLING_EXPENSE', 'FP_UPDATED_AT', 'FP_NOTIFY_READ_AT', 'DELIVERY_SCHEDULE_FPS', 'IS_FP_DELIVERY', 'THANKS', 'THX_UPDATED_AT', 'THX_NOTIFY_READ_AT', 'TEL_DATE', 'TEL_TIME_ZONE_0', 'TEL_TIME_ZONE_1', 'TEL_TIME_ZONE_2', 'TEL_TIME_ZONE_3', 'TEL_TIME_ZONE_4', 'TEL_TIME_ZONE_5', 'CUSTOMER_PHONE_NUMBER', 'TEL_UPDATED_AT', 'TEL_NOTIFY_READ_AT', 'MEETING_DATETIME_0', 'MEETING_DATETIME_1', 'MEETING_DATETIME_2', 'ONLINE_DESIRED', 'MTG_UPDATED_AT', 'MTG_NOTIFY_READ_AT', 'CREATED_AT', 'UPDATED_AT']


In [15]:
print(f"df_texts:{df_texts.columns.to_list()}")

df_texts:['USER_ID', 'ASSESSMENT_REPORT_ID', 'CONTENT', 'TXT_NOTIFY_READ_AT', 'CREATED_AT', 'UPDATED_AT']


# 各データの欠損率の確認 #

In [16]:
#欠損数・欠損率・欠損有無を確認する関数
def check_missing_info(df, columns=None):
    """
    Parameters:
    ----------
    df : 対象のデータフレーム
    columns : チェック対象のカラムリスト。None の場合は全カラム。
    ----------
    """
    # カラム指定がある場合のみ抽出
    target_df = df[columns] if columns is not None else df

    # 欠損情報の集計
    missing_info = target_df.isnull().sum().to_frame(name='欠損数')
    missing_info['全体件数'] = len(df)
    missing_info['欠損率(%)'] = (missing_info['欠損数'] / missing_info['全体件数'] * 100).round(2)
    missing_info['欠損有無'] = missing_info['欠損数'] > 0
    missing_info = missing_info[['欠損数', '全体件数', '欠損率(%)', '欠損有無']]

    return missing_info


In [17]:
with pd.option_context('display.max_rows', None):
    display(check_missing_info(df_reports_recent))

,欠損数,全体件数,欠損率(%),欠損有無
ID,0,206788,0.00,False
USER_ID,0,206788,0.00,False
IS_DEAL,0,206788,0.00,False
PROPERTY_KIND,0,206788,0.00,False
APARTMENT_ID,93757,206788,45.34,True
FLOOR_NUMBER,93854,206788,45.39,True
OWNER_SPACE,93757,206788,45.34,True
LAND_SPACE,113031,206788,54.66,True
BUILDING_SPACE,137691,206788,66.59,True
LAND_AND_BUILDING_SPACE,123147,206788,59.55,True


In [18]:
with pd.option_context('display.max_rows', None):
    display(check_missing_info(df_reads))

,欠損数,全体件数,欠損率(%),欠損有無
ASSESSMENT_REPORT_ID,0,54993,0.00,False
FIRST_READ_AT,37,54993,0.07,True
LAST_READ_AT,37,54993,0.07,True
PREVIOUS_LAST_READ_AT,28358,54993,51.57,True
IS_SEND_FIRST_SP_OPEN,0,54993,0.00,False
IS_SEND_SP_REOPEN,0,54993,0.00,False
READ_COUNT,0,54993,0.00,False
CREATED_AT,0,54993,0.00,False
UPDATED_AT,0,54993,0.00,False


In [19]:
with pd.option_context('display.max_rows', None):
    display(check_missing_info(df_inquiries))

,欠損数,全体件数,欠損率(%),欠損有無
USER_ID,0,83097,0.00,False
ASSESSMENT_REPORT_ID,0,83097,0.00,False
PRICE_DETAIL,0,83097,0.00,False
MEDIATION_BUY,0,83097,0.00,False
MARKET_CONDITION,0,83097,0.00,False
CONSULTATION_MOVE,0,83097,0.00,False
TAX_ADVICE,0,83097,0.00,False
SELLING_EXPENSE,0,83097,0.00,False
FP_UPDATED_AT,81070,83097,97.56,True
FP_NOTIFY_READ_AT,0,83097,0.00,False


In [20]:
with pd.option_context('display.max_rows', None):
    display(check_missing_info(df_texts))

,欠損数,全体件数,欠損率(%),欠損有無
USER_ID,0,512,0.0,False
ASSESSMENT_REPORT_ID,0,512,0.0,False
CONTENT,0,512,0.0,False
TXT_NOTIFY_READ_AT,0,512,0.0,False
CREATED_AT,0,512,0.0,False
UPDATED_AT,0,512,0.0,False


# データの結合 #


## 各keyの重複件数の確認 ##

In [21]:
def check_key_uniqueness(df, key_name, df_name="df"):
    dup_count = df[key_name].value_counts()
    print(f"{df_name}[{key_name}]：ユニークか？ {df[key_name].is_unique}")
    print(f"  重複値の件数（2件以上）: {(dup_count >= 2).sum()}")
    print(f"  最大出現回数: {dup_count.max()}")
    print("-" * 40)

In [22]:
check_key_uniqueness(df_reports_recent, 'ID', "df_reports_recent")
check_key_uniqueness(df_inquiries, 'ASSESSMENT_REPORT_ID', "df_inquiries")
check_key_uniqueness(df_texts, 'ASSESSMENT_REPORT_ID', "df_texts")
check_key_uniqueness(df_reads, 'ASSESSMENT_REPORT_ID', "df_reads")

df_reports_recent[ID]：ユニークか？ True
  重複値の件数（2件以上）: 0
  最大出現回数: 1
----------------------------------------
df_inquiries[ASSESSMENT_REPORT_ID]：ユニークか？ False
  重複値の件数（2件以上）: 46
  最大出現回数: 3
----------------------------------------
df_texts[ASSESSMENT_REPORT_ID]：ユニークか？ False
  重複値の件数（2件以上）: 36
  最大出現回数: 3
----------------------------------------
df_reads[ASSESSMENT_REPORT_ID]：ユニークか？ False
  重複値の件数（2件以上）: 32
  最大出現回数: 2
----------------------------------------


## 結合 ##

### STEP1 ###

絞り込み後のASSESSMENT＿REPORTSとASSESSMENT_REPORT_SP_READ_HISTORIESをマージ

In [23]:
# df_reports_recent と df_reads をマージ
df_with_reads_dup = df_reports_recent.merge(
    df_reads,
    left_on='ID',
    right_on='ASSESSMENT_REPORT_ID',
    how='left',
    suffixes=('', '_read')
)

# 結合できたかのフラグ追加
df_with_reads_dup['join_read'] = df_with_reads_dup['ASSESSMENT_REPORT_ID'].notna()

#ID重複を削除
df_with_reads = df_with_reads_dup.drop_duplicates(subset='ID', keep='first')

# 件数確認ログ出力
print("【ステップ①: df_reports_recent × df_reads】")
print(f"df_reports_recent 件数: {len(df_reports_recent)}")
print(f"df_reads 件数: {len(df_reads)}")
print(f"df_with_reads_dup.shape: {df_with_reads_dup.shape}")
print(f"df_with_reads.shape: {df_with_reads.shape}")
print("-" * 60)

【ステップ①: df_reports_recent × df_reads】
df_reports_recent 件数: 206788
df_reads 件数: 54993
df_with_reads_dup.shape: (206818, 318)
df_with_reads.shape: (206788, 318)
------------------------------------------------------------


In [24]:
df_with_reads
df_m = df_with_reads[df_with_reads['PROPERTY_KIND'] == 1]
print(df_m.shape)
df_with_reads['PROPERTY_KIND'].unique()


(113031, 318)


array([1., 2., 3.])

### STEP2 ###

ASSESSMENT_REPORT_INQUIRY_TEXTSをマージ

In [25]:
# df_with_reads と df_texts をマージ
df_with_texts_dup = df_with_reads.merge(
    df_texts,
    left_on='ID',
    right_on='ASSESSMENT_REPORT_ID',
    how='left',
    suffixes=('', '_txt')
)

# 結合できたかのフラグ追加
df_with_texts_dup['join_text'] = df_with_texts_dup['ASSESSMENT_REPORT_ID_txt'].notna()

# 1) CONTENTの有無でフラグ化（NaN・空文字・空白のみを0、それ以外を1）
df_with_texts_dup['CONTENT_FLG'] = (
    df_with_texts_dup['CONTENT'].fillna('').astype(str).str.strip().ne('')
).astype(int)

# 2) IDごとにフラグを合計して CONTENT_COUNT を作成しつつ、他のカラムは最初の値を保持
agg_dict = {
    'CONTENT_FLG': 'sum',  # フラグは合計
    'TXT_NOTIFY_READ_AT': 'first'  # フリー入力通知　未読/既読フラグは最初の値を使用
}

# その他全てのカラムは最初の値を保持
for col in df_with_texts_dup.columns:
    if col not in agg_dict and col != 'ID':
        agg_dict[col] = 'first'

# データ型を確認してから集計
numeric_cols = df_with_texts_dup.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if col not in agg_dict and col != 'ID':
        agg_dict[col] = 'first'

df_with_texts = df_with_texts_dup.groupby('ID', as_index=False).agg(agg_dict)
df_with_texts = df_with_texts.rename(columns={'CONTENT_FLG': 'CONTENT_COUNT'})

# 件数確認ログ出力
print("【ステップ②: df_with_reads × df_texts】")
print(f"df_with_reads 件数: {len(df_with_reads)}")
print(f"df_texts 件数: {len(df_texts)}")
print(f"df_with_texts_dup.shape: {df_with_texts_dup.shape}")
print(f"df_with_texts.shape: {df_with_texts.shape}")
print(f"PROPERTY_KIND unique values: {df_with_texts['PROPERTY_KIND'].unique()}")
print("-" * 60)

【ステップ②: df_with_reads × df_texts】
df_with_reads 件数: 206788
df_texts 件数: 512
df_with_texts_dup.shape: (206815, 326)
df_with_texts.shape: (206788, 326)
PROPERTY_KIND unique values: [1. 2. 3.]
------------------------------------------------------------


In [26]:
df_m = df_with_texts_dup[df_with_texts_dup['PROPERTY_KIND'] == 1]
print(df_m.shape)

(113045, 326)


In [27]:
df_m = df_with_texts[df_with_texts['PROPERTY_KIND'] == 1]
print(df_m.shape)

(113031, 326)


In [28]:
print(df_with_texts.columns.to_list())

['ID', 'CONTENT_COUNT', 'TXT_NOTIFY_READ_AT', 'USER_ID', 'IS_DEAL', 'PROPERTY_KIND', 'APARTMENT_ID', 'FLOOR_NUMBER', 'OWNER_SPACE', 'LAND_SPACE', 'BUILDING_SPACE', 'LAND_AND_BUILDING_SPACE', 'DIRECTION', 'ASSESSED_ON', 'CUSTOMER_NAME', 'PROPERTY_NAME', 'STAFF_NAME', 'COMPANY_DESCRIPTION', 'EMAIL', 'TEL', 'LINE_URL', 'HOMEPAGE', 'COVER_ASSESSED_ON', 'COVER_TITLE', 'TITLE_FONT_SIZE', 'COVER_CUSTOMER_NAME', 'COVER_STAFF_AND_COMPANY_NAME', 'COVER_DESCRIPTION', 'COVER_EMAIL', 'COVER_TEL', 'COVER_HOMEPAGE', 'COVER_STAFF_ROUND_IMAGE', 'COVER_STAFF_ROUND_IMAGE_COMMENT', 'ABOUT_PROPERTY_NAME', 'ABOUT_ADDRESS', 'LATITUDE', 'LONGITUDE', 'C_ID', 'ABOUT_TRAFFIC', 'ABOUT_MANAGER', 'ABOUT_LAND_PRIVILEGE', 'ABOUT_REGISTRY_GROUND', 'ABOUT_CITY_PLAN', 'ABOUT_TERRAIN', 'ABOUT_LAND_SHAPE', 'ABOUT_SETBACK_SPACE', 'ABOUT_CURRENT_STATE', 'ABOUT_RECONSTRUCT_DISABLED', 'ABOUT_USAGE_AREA', 'ABOUT_STRUCTURE', 'STRUCTURE_ID', 'ABOUT_BUILT_IN', 'ABOUT_FLOOR', 'ABOUT_BUILDING_COVERAGE', 'ABOUT_CONNECT_ROAD_STATE', 

### STEP3 ###

ASSESSMENT_REPORT_INQUIRIESをマージ

In [29]:
# df_with_texts と df_inquiries をマージ
# --- 前処理（そのまま） ---
df_all_dup = df_with_texts.merge(
    df_inquiries,
    left_on='ID',
    right_on='ASSESSMENT_REPORT_ID',
    how='left',
    suffixes=('', '_inq')
)

# 結合できたかフラグ
df_all_dup['join_inquiry'] = df_all_dup['ASSESSMENT_REPORT_ID_inq'].notna()

# ① 電話番号あり（10桁より大）→ 1、それ以外→ 0
if 'CUSTOMER_PHONE_NUMBER' in df_all_dup.columns:
    # 数字以外（ハイフン・空白・括弧など）を除去
    digits = (
        df_all_dup['CUSTOMER_PHONE_NUMBER']
        .fillna('')
        .astype(str)
        .str.replace(r'\D', '', regex=True)
    )
    df_all_dup['HOPE_TEL_FLG'] = (digits.str.len() >= 10).astype(int)
else:
    df_all_dup['HOPE_TEL_FLG'] = 0

# ② meeting のどれかに値があれば 1
meeting_cols = ['MEETING_DATETIME_0', 'MEETING_DATETIME_1', 'MEETING_DATETIME_2']
for c in meeting_cols:
    if c in df_all_dup.columns:
        df_all_dup[c] = pd.to_datetime(df_all_dup[c], errors='coerce')
df_all_dup['MEETING_FLG'] = df_all_dup[meeting_cols].notna().any(axis=1).astype(int)

# 0/1/NaN列を数値化
bin_cols = [
    'PRICE_DETAIL', 'MEDIATION_BUY', 'MARKET_CONDITION',
    'CONSULTATION_MOVE', 'TAX_ADVICE', 'SELLING_EXPENSE', 'THANKS',
    'THX_NOTIFY_READ_AT', 'TEL_NOTIFY_READ_AT', 'MTG_NOTIFY_READ_AT'
]
present_bin_cols = [c for c in bin_cols if c in df_all_dup.columns]
if present_bin_cols:
    for col in present_bin_cols:
        df_all_dup[col] = pd.to_numeric(df_all_dup[col], errors='coerce').fillna(0).astype(int)

# --- 集約設定 ---
sum_cols = present_bin_cols + ['HOPE_TEL_FLG', 'MEETING_FLG']

agg_dict = {col: 'sum' for col in sum_cols}
agg_dict['join_inquiry'] = 'any'  # どれか1つでも結合できていれば True

# 代表行を“最新優先”で並べてから集約
for c in ['CREATED_AT', 'UPDATED_AT', 'CREATED_AT_inq', 'UPDATED_AT_inq']:
    if c in df_all_dup.columns:
        df_all_dup[c] = pd.to_datetime(df_all_dup[c], errors='coerce')

df_all_dup = df_all_dup.sort_values(
    by=[c for c in ['CREATED_AT', 'UPDATED_AT', 'CREATED_AT_inq', 'UPDATED_AT_inq'] if c in df_all_dup.columns],
    ascending=False
)

# それ以外は first を採用（最新優先で並べたので first=最新）
for col in df_all_dup.columns:
    if col not in agg_dict and col != 'ID':
        agg_dict[col] = 'first'

# --- 集約（これで「合計 + 他カラム保持」）---
df_all = df_all_dup.groupby('ID', as_index=False).agg(agg_dict)

# 合計列を *_COUNT に改名
df_all = df_all.rename(columns={c: f'{c}_COUNT' for c in sum_cols})

# ログ
print("【ステップ③: df_with_texts × df_inquiries】")
print(f"df_with_texts 件数: {len(df_with_texts)}")
print(f"df_inquiries 件数: {len(df_inquiries)}")
print(f"df_all_dup.shape: {df_all_dup.shape}")
print(f"df_all.shape: {df_all.shape}")
print("-" * 60)


【ステップ③: df_with_texts × df_inquiries】
df_with_texts 件数: 206788
df_inquiries 件数: 83097
df_all_dup.shape: (206814, 362)
df_all.shape: (206788, 362)
------------------------------------------------------------


In [30]:
df_m = df_all[df_all['PROPERTY_KIND'] == 1]
print(df_m.shape)

(113031, 362)


In [31]:
print(df_all_dup.columns.to_list())

['ID', 'CONTENT_COUNT', 'TXT_NOTIFY_READ_AT', 'USER_ID', 'IS_DEAL', 'PROPERTY_KIND', 'APARTMENT_ID', 'FLOOR_NUMBER', 'OWNER_SPACE', 'LAND_SPACE', 'BUILDING_SPACE', 'LAND_AND_BUILDING_SPACE', 'DIRECTION', 'ASSESSED_ON', 'CUSTOMER_NAME', 'PROPERTY_NAME', 'STAFF_NAME', 'COMPANY_DESCRIPTION', 'EMAIL', 'TEL', 'LINE_URL', 'HOMEPAGE', 'COVER_ASSESSED_ON', 'COVER_TITLE', 'TITLE_FONT_SIZE', 'COVER_CUSTOMER_NAME', 'COVER_STAFF_AND_COMPANY_NAME', 'COVER_DESCRIPTION', 'COVER_EMAIL', 'COVER_TEL', 'COVER_HOMEPAGE', 'COVER_STAFF_ROUND_IMAGE', 'COVER_STAFF_ROUND_IMAGE_COMMENT', 'ABOUT_PROPERTY_NAME', 'ABOUT_ADDRESS', 'LATITUDE', 'LONGITUDE', 'C_ID', 'ABOUT_TRAFFIC', 'ABOUT_MANAGER', 'ABOUT_LAND_PRIVILEGE', 'ABOUT_REGISTRY_GROUND', 'ABOUT_CITY_PLAN', 'ABOUT_TERRAIN', 'ABOUT_LAND_SHAPE', 'ABOUT_SETBACK_SPACE', 'ABOUT_CURRENT_STATE', 'ABOUT_RECONSTRUCT_DISABLED', 'ABOUT_USAGE_AREA', 'ABOUT_STRUCTURE', 'STRUCTURE_ID', 'ABOUT_BUILT_IN', 'ABOUT_FLOOR', 'ABOUT_BUILDING_COVERAGE', 'ABOUT_CONNECT_ROAD_STATE', 

# 追加カラム作成 #

In [32]:
# アクセス有無フラグの作成
df_all['READ_FLG'] = (
    pd.to_datetime(df_all['FIRST_READ_AT'], errors='coerce')
      .notna()
      .astype(int)
)

In [33]:
#アクセス日数差分のカラムを作成
# 日付型に変換（必要な場合）
df_all['FIRST_READ_AT'] = pd.to_datetime(df_all['FIRST_READ_AT'], errors='coerce')
df_all['LAST_READ_AT'] = pd.to_datetime(df_all['LAST_READ_AT'], errors='coerce')
df_all['PREVIOUS_LAST_READ_AT'] = pd.to_datetime(df_all['PREVIOUS_LAST_READ_AT'], errors='coerce')

# 日数差分（負の値もそのまま）
df_all['LAST_FIRST_TIME'] = (df_all['LAST_READ_AT'] - df_all['FIRST_READ_AT']).dt.days
df_all['LAST_PREVIOS_TIME'] = (df_all['LAST_READ_AT'] - df_all['PREVIOUS_LAST_READ_AT']).dt.days
print(df_all.shape)

(206788, 365)


In [34]:
#アクセス日数差分のカラムを作成
# 日付型に変換（必要な場合）
df_all['FIRST_READ_AT'] = pd.to_datetime(df_all['FIRST_READ_AT'], errors='coerce')
df_all['LAST_READ_AT'] = pd.to_datetime(df_all['LAST_READ_AT'], errors='coerce')
df_all['PREVIOUS_LAST_READ_AT'] = pd.to_datetime(df_all['PREVIOUS_LAST_READ_AT'], errors='coerce')

# 日数差分（負の値もそのまま）
df_all['LAST_FIRST_TIME'] = (df_all['LAST_READ_AT'] - df_all['FIRST_READ_AT']).dt.days
df_all['LAST_PREVIOS_TIME'] = (df_all['LAST_READ_AT'] - df_all['PREVIOUS_LAST_READ_AT']).dt.days
print(df_all.shape)

(206788, 365)


In [35]:
# 訪問査定ありフラグの作成
#０でも値があると判定するので注意
print(df_all['ASSESS_RENT_PRICE'].unique())
df_all['RENT_PRICE_FLG'] = (
    pd.to_numeric(df_all['ASSESS_RENT_PRICE'], errors='coerce')
      .notna().astype(int)
)
print(df_all.shape)

['16.0〜17.0' '9.0〜12.0' None ... '17.0〜35.0' '28.5〜47.2' '27.0〜42.8']
(206788, 366)


In [36]:
#賃料査定有無フラグの作成
print(df_all['VISIT_ASSESSMENT_PRICE'].unique())

#文字列をNANに変換し、フラグ化
df_all['VISIT_ASSESSMENT_FLG'] = (
    pd.to_numeric(df_all['VISIT_ASSESSMENT_PRICE'], errors='coerce')
      .notna().astype(int)
)
print(df_all.shape)

[None '2222' '13400' '12000' '1700' '7000' '訪問査定 ご依頼待ち' '2780' '訪問査定後ご提示'
 '室内拝見後' '1790' '1200' '4400' '5780' '2280〜1980' '10850' '2030' '2100'
 '1780' '5700' '18,500' '１３２００' '5000' '１３３００' '未定' '3390' '9000' '2170'
 '5,000' '1,780' '2,011' '2400' '2,680～2,780' '内見後' '2180' '2050' '1575'
 '3150' '3500' '4,820' '600' '5100～5200' '5480' '4980' '4780'
 '訪問査定後\u3000ご提示' '3900' '1460' '850' '1080' '3180' '1980～2180' '4580'
 '2,380万円～2,580万円' '2700' '前ページ参照' '6700～6900' '5280' '2420' '5580' '1680'
 '4,490' '2513' '4,480' '4800' '6,990~7,190' '480-580' '未査定' '2800～3300'
 '1980' '2,300' '未訪問' '2,480']
(206788, 367)


In [37]:
#金利についてのコメント有無フラグの作成

cols = [
    'FLOATING_INTEREST_COMMENT',
    'TEN_YEARS_FIXED_INTEREST_COMMENT',
    'ALL_YEARS_FIXED_INTEREST_COMMENT'
]

# 各列を：NaN→'' → 文字列化 → 前後空白除去
comments = df_all[cols].fillna('').astype(str).apply(lambda s: s.str.strip())

# 3列のうち1つでも1文字以上あれば 1、全て空なら 0
df_all['INTEREST_COMMENT_FLG'] = (comments != '').any(axis=1).astype(int)
print(df_all.shape)

(206788, 368)


In [38]:
df_m = df_all[df_all['PROPERTY_KIND'] == 1]
print(df_m.shape)

(113031, 368)


# 保存 #

In [39]:
df_all.to_csv(os.path.join(OUTPUT_DIR, "1.ASSESSMENT_REPORTS_MERGE.csv"), index=False)

# 結合後データの確認 #

In [40]:

# 表示設定を変更
pd.set_option('display.max_columns', None)
df_all.head(3)

,ID,PRICE_DETAIL_COUNT,MEDIATION_BUY_COUNT,MARKET_CONDITION_COUNT,CONSULTATION_MOVE_COUNT,TAX_ADVICE_COUNT,SELLING_EXPENSE_COUNT,THANKS_COUNT,THX_NOTIFY_READ_AT_COUNT,TEL_NOTIFY_READ_AT_COUNT,MTG_NOTIFY_READ_AT_COUNT,HOPE_TEL_FLG_COUNT,MEETING_FLG_COUNT,join_inquiry,CONTENT_COUNT,TXT_NOTIFY_READ_AT,USER_ID,IS_DEAL,PROPERTY_KIND,APARTMENT_ID,FLOOR_NUMBER,OWNER_SPACE,LAND_SPACE,BUILDING_SPACE,LAND_AND_BUILDING_SPACE,DIRECTION,ASSESSED_ON,CUSTOMER_NAME,PROPERTY_NAME,STAFF_NAME,COMPANY_DESCRIPTION,EMAIL,TEL,LINE_URL,HOMEPAGE,COVER_ASSESSED_ON,COVER_TITLE,TITLE_FONT_SIZE,COVER_CUSTOMER_NAME,COVER_STAFF_AND_COMPANY_NAME,COVER_DESCRIPTION,COVER_EMAIL,COVER_TEL,COVER_HOMEPAGE,COVER_STAFF_ROUND_IMAGE,COVER_STAFF_ROUND_IMAGE_COMMENT,ABOUT_PROPERTY_NAME,ABOUT_ADDRESS,LATITUDE,LONGITUDE,C_ID,ABOUT_TRAFFIC,ABOUT_MANAGER,ABOUT_LAND_PRIVILEGE,ABOUT_REGISTRY_GROUND,ABOUT_CITY_PLAN,ABOUT_TERRAIN,ABOUT_LAND_SHAPE,ABOUT_SETBACK_SPACE,ABOUT_CURRENT_STATE,ABOUT_RECONSTRUCT_DISABLED,ABOUT_USAGE_AREA,ABOUT_STRUCTURE,STRUCTURE_ID,ABOUT_BUILT_IN,ABOUT_FLOOR,ABOUT_BUILDING_COVERAGE,ABOUT_CONNECT_ROAD_STATE,ABOUT_CONNECT_ROAD,ABOUT_CAPACITY_RATIO,ABOUT_BUILDING_COVERAGE_AND_CAPACITY_RATIO,ABOUT_UNIT_AMOUNT,ABOUT_SELLER,ABOUT_CONSTRUCTOR,ABOUT_REMARKS,ASSESS_SELL_TITLE,ASSESS_SELL_PRICE,ASSESS_SELL_SQUARE_PRICE,ASSESS_SELL_TSUBO_PRICE,ASSESS_SELL_TERM,ASSESS_SELL_PRICE_FROM,ASSESS_SELL_PRICE_TO,ASSESS_SELL_SELF_STORY_COUNT,ASSESS_SELL_SIMILAR_STORY_COUNT,ASSESS_SELL_BUILDING_PRICE,ASSESS_SELL_BUILDING_UNIT_PRICE,ASSESS_SELL_HOUSE_LAND_PRICE,ASSESS_SELL_COMMENT,ASSESS_HOUSE_COMMENT,ASSESS_SELL_PAGE_BREAK,INDICATE_VISIT_ASSESSMENT,VISIT_ASSESSMENT_PRICE,VISIT_ASSESSMENT_COMMENT,VISIT_ASSESSMENT_MEMO,VISIT_ASSESSMENT_TITLE,ASSESS_SUGGEST_PRICE,ASSESS_SUGGEST_COMMENT,ASSESS_SUGGEST_PAGE_BREAK,CHALLENGE1_TITLE,CHALLENGE1_PRICE,CHALLENGE1_TSUBO_PRICE,CHALLENGE1_TERM,CHALLENGE1_COMMENT,CHALLENGE1_PAGE_BREAK,CHALLENGE2_TITLE,CHALLENGE2_PRICE,CHALLENGE2_TSUBO_PRICE,CHALLENGE2_TERM,CHALLENGE2_COMMENT,CHALLENGE2_PAGE_BREAK,CHALLENGE3_TITLE,CHALLENGE3_PRICE,CHALLENGE3_TSUBO_PRICE,CHALLENGE3_TERM,CHALLENGE3_COMMENT,CHALLENGE3_PAGE_BREAK,ASSESS_PURCHASE_PRICE,ASSESS_PURCHASE_TERM,ASSESS_PURCHASE_COMMENT,ASSESS_PURCHASE_PAGE_BREAK,ASSESS_RENT_TITLE,ASSESS_RENT_PRICE,ASSESS_RENT_YIELD,ASSESS_RENT_COMMENT,ASSESS_RENT_PAGE_BREAK,INDICATE_DEMOLITION_COST,DEMOLITION_P_NAME,DEMOLITION_ABOUT_STRUCTURE,DEMOLITION_BUILDING_SPACE_TSUBO,DEMOLITION_ROAD_WIDTH,DEMOLITION_HOUSE_DISTANCE,DEMOLITION_COST,DEMOLITION_MIN_COST,DEMOLITION_MAX_COST,COVER_STATISTICS_TITLE,COVER_MARKET_TITLE,ASSESS_VISIT_PAGE_BREAK,ASSESS_STAFF_COMMENT,ASSESS_STAFF_PAGE_BREAK,AREA_HUMAN_COMMENT,SELL_FLOW1_TITLE,SELL_FLOW1_DETAIL,SELL_FLOW2_TITLE,SELL_FLOW2_DETAIL,SELL_FLOW3_TITLE,SELL_FLOW3_DETAIL,SELL_FLOW4_TITLE,SELL_FLOW4_DETAIL,SELL_FLOW5_TITLE,SELL_FLOW5_DETAIL,SELL_FLOW6_TITLE,SELL_FLOW6_DETAIL,SELL_FLOW7_TITLE,SELL_FLOW7_DETAIL,LAST_COMMENT,LAST_STAFF_IMAGE,LAST_STAFF_NAME,LAST_TEL,LAST_EMAIL,LAST_PROFILE,INDICATE_COVER_ASSESSED_ON,INDICATE_COVER_LOGO,INDICATE_COVER_CUSTOMER_NAME,INDICATE_COVER_ABOUT,INDICATE_COVER_STAFF_AND_COMPANY_NAME,INDICATE_COVER_DESCRIPTION,INDICATE_COVER_EMAIL,INDICATE_COVER_TEL,INDICATE_COVER_HOMEPAGE,INDICATE_COVER_STAFF_ROUND_IMAGE,INDICATE_STAFF_ROUND_IMAGE,INDICATE_COVER_STAFF_ROUND_IMAGE_COMMENT,INDICATE_ABOUT_INFO,INDICATE_ABOUT_EXCLUSIVE_INFO,INDICATE_ABOUT_MAP,INDICATE_ASSESS_VALUE,INDICATE_ASSESS_SELL,INDICATE_ASSESS_SELL_RANGE,INDICATE_ASSESS_SELL_COUNT,INDICATE_ASSESS_TK_BREAKDOWN,INDICATE_ASSESS_SUGGEST,INDICATE_ASSESS_PURCHASE,INDICATE_ASSESS_RENT,INDICATE_ASSESS_PROPERTY_INFO,INDICATE_ASSESS_VISIT,INDICATE_ASSESS_STAFF_COMMENT,INDICATE_SELL_STORY_SELF,INDICATE_SELL_STORY_SIMILAR,INDICATE_RENT_STORY_SELF,INDICATE_RENT_STORY_SIMILAR,INDICATE_COVER_STATISTICS,INDICATE_MARKET_REPORTS,INDICATE_PRICE_HISTORY,INDICATE_PRICE_HISTORY_GRAPH,INDICATE_PRICE_HISTORY_SELF_COMPARE,INDICATE_PRICE_HISTORY_AREA_COMPARE,INDICATE_AREA_HUMAN,INDICATE_AREA_HUMAN_AGE_GRAPH,INDICATE_AR

# 結合後データの欠損率の確認 #

In [41]:
with pd.option_context('display.max_rows', None):
    display(check_missing_info(df_all))

,欠損数,全体件数,欠損率(%),欠損有無
ID,0,206788,0.00,False
PRICE_DETAIL_COUNT,0,206788,0.00,False
MEDIATION_BUY_COUNT,0,206788,0.00,False
MARKET_CONDITION_COUNT,0,206788,0.00,False
CONSULTATION_MOVE_COUNT,0,206788,0.00,False
TAX_ADVICE_COUNT,0,206788,0.00,False
SELLING_EXPENSE_COUNT,0,206788,0.00,False
THANKS_COUNT,0,206788,0.00,False
THX_NOTIFY_READ_AT_COUNT,0,206788,0.00,False
TEL_NOTIFY_READ_AT_COUNT,0,206788,0.00,False
